In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings_df = pd.read_csv('/data/kakao_data/ratings_two.csv')
ratings_df = ratings_df[['cafe','user_id','rating']]

In [ ]:
print(len(ratings_df))

23308


In [ ]:
print(ratings_df['user_id'].nunique())
print(ratings_df['cafe'].nunique())

6842
5165


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder1 = LabelEncoder()
ratings_df['cafe']=encoder1.fit_transform(ratings_df['cafe'])
encoder2 = LabelEncoder()
ratings_df['user_id']=encoder2.fit_transform(ratings_df['user_id'])

# package를 사용하기 위해서는 문자열이 아닌 숫자로 이루어져 있어야 함

In [ ]:
ratings_df.head()

,cafe,user_id,rating
0,22,0,5.0
1,3677,0,5.0
2,239,1,1.0
3,4640,1,1.0
4,4871,1,5.0


In [ ]:
ratings_df.to_csv('경로지정/ratings_for_als_two.csv',index=False)

# pyspark 설치 및 환경 설정

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 30 kB/s 
     |████████████████████████████████| 199 kB 57.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f2cfcb4fade1aa61efd04e24d8c76ecf5739d1e6d8440e1168a3c2f8ceb3c210
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.0-bin-hadoop2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"



# install findspark using pip
!pip install -q findspark

--2022-10-02 07:26:55--  https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz’

spark-3.3.0-bin-had 100%[===================>] 261.38M  26.5MB/s    in 10s     

2022-10-02 07:27:06 (25.5 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz’ saved [274079476/274079476]



In [ ]:
#Setting up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop2"

In [ ]:
#Initialize Spark session using findspark lib
import findspark

findspark.init()


In [ ]:
findspark.find()

'/content/spark-3.3.0-bin-hadoop2'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#setting the path of the files
ratings_file ='경로지정/ratings_for_als_two.csv'

In [ ]:
def readFiles(filename):
  data = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load(filename,header=True)
  return data

In [ ]:
#Read the data files
ratings = readFiles(ratings_file)

In [ ]:
ratings.show(5)

+----+-------+------+
|cafe|user_id|rating|
+----+-------+------+
|  22|      0|   5.0|
|3677|      0|   5.0|
| 239|      1|   1.0|
|4640|      1|   1.0|
|4871|      1|   5.0|
+----+-------+------+
only showing top 5 rows



In [ ]:
#We will consider ratings file as the file of interest as it has the rating values
#checking the schema
ratings.printSchema()

root
 |-- cafe: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: double (nullable = true)



In [ ]:
#Data dimensions
print('No. of rows: %d' % ratings.count())
ratings.show(5)

No. of rows: 23308
+----+-------+------+
|cafe|user_id|rating|
+----+-------+------+
|  22|      0|   5.0|
|3677|      0|   5.0|
| 239|      1|   1.0|
|4640|      1|   1.0|
|4871|      1|   5.0|
+----+-------+------+
only showing top 5 rows



# model train/test

In [ ]:
# Randomly split the data into train and test where 80% data is in train and remaining is test
train, test = ratings.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
# Build a recommendation model using Alternating Least Squares method
# Evaluate the model by computing the RMSE on the test data
model = ALS(userCol="user_id", itemCol="cafe", ratingCol="rating", nonnegative=True, coldStartStrategy="drop").fit(train)



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [ ]:
# Make predictions and print the RMSE of the ALS model
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("New RMSE: ", evaluator.evaluate(model.transform(test)))

New RMSE:  1.7930604499251805


# Implementing ALS with cross Validation

In [ ]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

In [ ]:
# we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

model = ALS(userCol="user_id", itemCol="cafe", ratingCol="rating", nonnegative = True, coldStartStrategy="drop")

#For Parameter tuning of the ALS model we use ParamGridBuilder function
#We tune two parameters 
#1. The Regularization parameter ranging from 0.1, 0.01, 0.001, 0.0001
#2. The rank for matrix factorization
paramGrid = ParamGridBuilder() \
    .addGrid(model.regParam, [0.1, 0.05, 0.01, 0.001]) \
    .addGrid(model.rank, [5, 10, 20, 30]) \
    .build()

#Defining a cross-validator object
#Setting up CV and adding parameters. We will be performing a 5 fold CV
crossvalidation = CrossValidator(estimator = model,
                     estimatorParamMaps = paramGrid,
                     evaluator = evaluator,
                     numFolds=5)

In [ ]:
# Run cross-validation, and choose the best set of parameters.
Best_model = crossvalidation.fit(train).bestModel

In [ ]:
#The Best_model
print(type(Best_model))
#Complete the code below to extract the ALS model parameters
print("**Best Model**")
#Rank
print("Rank: ", Best_model._java_obj.parent().getRank())
#MaxIter
print("MaxIter: ", Best_model._java_obj.parent().getMaxIter())
#RegParam
print("RegParam: ", Best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
Rank:  30
MaxIter:  10
RegParam:  0.1


In [ ]:
# Calculate the RMSE on test data using the best set of parameters obtained after cross validation
print("Best RMSE value is: ", evaluator.evaluate(Best_model.transform(test)))

Best RMSE value is:  1.768466808587618


In [ ]:
pred = Best_model.transform(test)
pred.show(10)

+----+-------+------+----------+
|cafe|user_id|rating|prediction|
+----+-------+------+----------+
|4935|   4307|   1.0| 0.9019609|
|4935|    673|   4.0| 3.3105643|
|1342|   4773|   5.0| 4.0692058|
|4900|   4172|   5.0| 0.8385615|
|3175|   3611|   5.0| 3.9506624|
|1829|   2842|   4.0| 1.8070987|
|4519|   6178|   5.0|   4.37834|
|1959|   3689|   4.0| 3.8196316|
|2142|   3813|   4.0| 2.5248132|
|4101|   4831|   1.0|0.35485193|
+----+-------+------+----------+
only showing top 10 rows



# Predict

In [ ]:
ratings.head()

Row(cafe=22, user_id=0, rating=5.0)

In [ ]:
new_model = ALS(userCol="user_id", itemCol="cafe", ratingCol="rating", nonnegative = True, coldStartStrategy="drop",
            rank=30, maxIter=10, regParam=0.1).fit(ratings)
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [ ]:
predictions=new_model.transform(ratings)
rmse=evaluator.evaluate(predictions)
print("New RMSE: ", evaluator.evaluate(new_model.transform(ratings)))

New RMSE:  0.14225627837624477


In [ ]:
predictions

DataFrame[cafe: int, user_id: int, rating: double, prediction: float]

In [ ]:
predictions.show(10)

+----+-------+------+----------+
|cafe|user_id|rating|prediction|
+----+-------+------+----------+
|1342|   3175|   5.0|  4.921738|
|5156|   6466|   2.0| 2.0594954|
|5156|   1322|   4.0| 3.8349748|
|4935|   4307|   1.0|0.98496115|
|4519|   2559|   4.0|  3.903322|
|4935|   2559|   4.0|  3.715771|
|3749|   4697|   5.0|  4.764438|
|4935|    673|   4.0|  4.017681|
|2122|   2294|   1.0| 1.1999673|
|1342|   4773|   5.0|  4.897755|
+----+-------+------+----------+
only showing top 10 rows



In [ ]:
for_an_user = predictions.where(predictions.user_id==6620)
for_an_user.show(5)

+----+-------+------+----------+
|cafe|user_id|rating|prediction|
+----+-------+------+----------+
|1060|   6620|   5.0| 4.9270782|
|5089|   6620|   5.0|  4.958515|
+----+-------+------+----------+



In [ ]:
movie_recommendation = Best_model.recommendForAllUsers(500)
movie_recommendation.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{1279, 5.034777}...|
|     12|[{5102, 4.866268}...|
|     22|[{298, 5.0480247}...|
|     26|[{730, 4.8772073}...|
|     27|[{2273, 4.95709},...|
|     28|[{913, 5.855925},...|
|     31|[{1388, 3.229459}...|
|     34|[{1873, 4.057873}...|
|     47|[{5010, 5.488736}...|
|     53|[{2873, 5.2557135...|
+-------+--------------------+
only showing top 10 rows



In [ ]:
movie_recommendation = movie_recommendation.toPandas()

In [ ]:
movie_recommendation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6431 entries, 0 to 6430
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          6431 non-null   int32 
 1   recommendations  6431 non-null   object
dtypes: int32(1), object(1)
memory usage: 75.5+ KB


In [ ]:
# pred_df_new['cafe'] = encoder1.inverse_transform(pred_df_new['cafe'])
movie_recommendation['user_id'] = encoder2.inverse_transform(movie_recommendation['user_id'])

In [ ]:
movie_recommendation.head()

,user_id,recommendations
0,100ck0j,"[(1279, 5.0347771644592285), (4871, 4.85388946..."
1,1021d6q,"[(5102, 4.866268157958984), (2658, 4.501917839..."
2,103e6a6,"[(298, 5.048024654388428), (4513, 4.8899378776..."
3,103m6ou,"[(730, 4.877207279205322), (4404, 4.0920867919..."
4,1040co5,"[(2273, 4.957089900970459), (3406, 4.932579517..."


In [ ]:
movie_recommendation.to_csv('/content/drive/MyDrive/modeling/study/승주/als_500_predictions_two.csv',index=False)

In [ ]:
movie_recommendation[movie_recommendation['user_id']=='10l9o2l']

,user_id,recommendations
4727,10l9o2l,"[(2566, 5.276892185211182), (2018, 5.206782341..."


# Recommender

## recommend를 위한 전처리

In [ ]:
df = pd.read_csv('저장경로/als_500_predictions_two.csv')
df.head()

,user_id,recommendations
0,100ck0j,"[Row(cafe=1279, rating=5.0347771644592285), Ro..."
1,1021d6q,"[Row(cafe=5102, rating=4.866268157958984), Row..."
2,103e6a6,"[Row(cafe=298, rating=5.048024654388428), Row(..."
3,103m6ou,"[Row(cafe=730, rating=4.877207279205322), Row(..."
4,1040co5,"[Row(cafe=2273, rating=4.957089900970459), Row..."


In [ ]:
len(df)

6431

In [ ]:
df['recommendations'][0]

'[Row(cafe=1279, rating=5.0347771644592285), Row(cafe=4871, rating=4.853889465332031), Row(cafe=3032, rating=4.356612205505371), Row(cafe=3245, rating=4.15034818649292), Row(cafe=1725, rating=4.091806888580322), Row(cafe=2819, rating=4.062037944793701), Row(cafe=2554, rating=4.019588470458984), Row(cafe=226, rating=3.979816436767578), Row(cafe=3530, rating=3.969014883041382), Row(cafe=2769, rating=3.9005813598632812), Row(cafe=2508, rating=3.892796277999878), Row(cafe=619, rating=3.8417153358459473), Row(cafe=2275, rating=3.82362961769104), Row(cafe=1453, rating=3.694737195968628), Row(cafe=67, rating=3.685147762298584), Row(cafe=3447, rating=3.6573615074157715), Row(cafe=2649, rating=3.637401819229126), Row(cafe=922, rating=3.618968963623047), Row(cafe=1388, rating=3.603224754333496), Row(cafe=1385, rating=3.601694345474243), Row(cafe=3783, rating=3.601694345474243), Row(cafe=3260, rating=3.5913240909576416), Row(cafe=2032, rating=3.54164457321167), Row(cafe=1699, rating=3.53659486770

In [ ]:
full = pd.DataFrame(columns = ['user_id','cafe','rating'])

for i in range(len(df)):
  rec = df['recommendations'][i]
  rec = rec.replace('[','')
  rec = rec.replace(' Row(cafe=','')
  rec = rec.replace(', rating=',',')
  rec = rec.replace(')','')
  rec = rec.replace(']','')
  rec = rec.replace('Row(cafe=','')

  lists = rec.split(',')
  cafe_list = [0 for k in range(500)]
  rating_list =  [0 for k in range(500)]
  userid = df['user_id'][i]
  for j in range(500):
    cafe_index = 2*j
    rating_index = 2*j + 1
    cafe_list[j] = int(lists[cafe_index])
    rating_list[j] = float(lists[rating_index])
    one_df = pd.DataFrame({'user_id':userid, 'cafe':cafe_list, 'rating':rating_list})
  full = full.append(one_df)
  if i%100==0:
    print(i, len(full))



0 500
100 50500
200 100500
300 150500
400 200500
500 250500
600 300500
700 350500
800 400500
900 450500
1000 500500
1100 550500
1200 600500
1300 650500
1400 700500
1500 750500
1600 800500
1700 850500
1800 900500
1900 950500
2000 1000500
2100 1050500
2200 1100500
2300 1150500
2400 1200500
2500 1250500
2600 1300500
2700 1350500
2800 1400500
2900 1450500
3000 1500500
3100 1550500
3200 1600500
3300 1650500
3400 1700500
3500 1750500
3600 1800500
3700 1850500
3800 1900500
3900 1950500
4000 2000500
4100 2050500
4200 2100500
4300 2150500
4400 2200500
4500 2250500
4600 2300500
4700 2350500
4800 2400500
4900 2450500
5000 2500500
5100 2550500
5200 2600500
5300 2650500
5400 2700500
5500 2750500
5600 2800500
5700 2850500
5800 2900500
5900 2950500
6000 3000500
6100 3050500
6200 3100500
6300 3150500
6400 3200500


In [ ]:
full['cafe'] = encoder1.inverse_transform(full['cafe'].astype(int))

In [ ]:
full.pivot(index='user_id', columns='cafe', values='rating')

cafe,044워리어스 당산동1가,10인치샌드위치&커피 봉천동,125coffee 갈현동,139COFFEE 전농동,1980벽돌집 신정동,1월의윤슬 내발산동,205도씨 명륜2가,212베이크샵 마곡동,22번가 신내동,24H스터디카페청춘 면목점 면목동,...,히어로보드게임카페 홍대1호점 서교동,히어로보드게임카페 홍대2호점 동교동,히어로스터 신도림동,히어커피 양재동,히자우 홍제동,히포커피 대학동점 신림동,히포커피서울대점 서울대점 신림동,히히냥냥 역삼동,힐브레드 마곡동,힘들땐마카롱 신당동
user_id,,,,,,,,,,,,,,,,,,,,,
100bln7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.148582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100ck0j,2.913923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.882658,2.957081,NaN,NaN,3.176812,NaN,NaN,NaN,NaN,NaN
100i22n,3.632898,NaN,NaN,NaN,NaN,3.635743,NaN,NaN,NaN,NaN,...,3.850483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100ooji,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.384001,NaN,NaN,NaN,NaN,NaN
100q04c,4.594968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.518830,4.435578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vv2c9h,NaN,NaN,NaN,2.956743,NaN,3.216803,NaN,2.785614,NaN,NaN,...,2.938166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vv73qf,NaN,NaN,NaN,NaN,NaN,3.333267,NaN,2.768569,NaN,NaN,...,NaN,2.643579,NaN,NaN,NaN,NaN,NaN,NaN,2.614352,NaN
vvcfbq,NaN,NaN,NaN,2.520373,NaN,NaN,NaN,NaN,NaN,NaN,...,2.483734,NaN,NaN,NaN,2.767430,NaN,NaN,NaN,NaN,NaN


In [ ]:
als_full = full.pivot(index='user_id', columns='cafe', values='rating')

## real recommend

In [ ]:
def recommend_cafe(df_als_preds, user_id, ori_ratings_df, num_recommendations=5):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_als_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']


    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')

    # 이미 갔던 곳 제외하기 위한 코드 추가
    user_data.reset_index(inplace=True, drop=True)
    for i in range(len(user_data)):
      recommendations = recommendations[recommendations.index!=user_data.loc[i,'cafe']]

    recommendations = pd.merge(recommendations, public_df.set_index('cafe')[['시군구명','도로명주소']], left_index=True,right_index=True,how='left')
    recommendations.drop('cafe',axis=1,inplace=True)

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [ ]:
def recommend_cafe_address(df_als_preds, user_id, ori_ratings_df, num_recommendations=5, address='영등포구'):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_als_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']


    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')

    # 이미 갔던 곳 제외하기 위한 코드 추가
    user_data.reset_index(inplace=True, drop=True)
    for i in range(len(user_data)):
      recommendations = recommendations[recommendations.index!=user_data.loc[i,'cafe']]
    
    recommendations = pd.merge(recommendations, public_df.set_index('cafe')[['시군구명','도로명주소']], left_index=True,right_index=True,how='left')
    recommendations.drop('cafe',axis=1,inplace=True)

    ## address 부분
    recommendations = recommendations[recommendations['시군구명']==address]

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [ ]:
already_rated, predictions = recommend_cafe(als_full, user_id='10l9o2l', ori_ratings_df=ratings_df,num_recommendations=10)

In [ ]:
predictions

,Predictions,시군구명,도로명주소
에그마카슈 건대점 화양동,5.276892,광진구,서울특별시 광진구 군자로 26
스타벅스 금호역점 금호동4가,5.206782,성동구,서울특별시 성동구 동호로 99
카페 바움758 청량리동,5.204337,동대문구,서울특별시 동대문구 회기로 36
토프레소 마장역점 마장동,5.171670,성동구,서울특별시 성동구 마장로 305
딮다 신수동,5.153128,마포구,서울특별시 마포구 광성로6길 34
앤디앤쌤 연희동,5.150989,서대문구,서울특별시 서대문구 연희로10길 19
바늘카페 연희동,5.119505,서대문구,서울특별시 서대문구 연희로11가길 15
두냉심열 봉천동,5.116771,관악구,서울특별시 관악구 인헌6길 9
요샌 봉천동,5.092241,관악구,서울특별시 관악구 남부순환로230길 42
빽다방 올림픽북단점 구의동,5.077830,광진구,서울특별시 광진구 아차산로 480


In [ ]:
already_rated, predictions = recommend_cafe_address(als_full, user_id='10l9o2l', ori_ratings_df=ratings_df,num_recommendations=10,
                                                    address='서대문구')

In [ ]:
predictions

,Predictions,시군구명,도로명주소
앤디앤쌤 연희동,5.150989,서대문구,서울특별시 서대문구 연희로10길 19
바늘카페 연희동,5.119505,서대문구,서울특별시 서대문구 연희로11가길 15
아로이커피 홍은벽산점 홍은동,5.034958,서대문구,서울특별시 서대문구 홍은중앙로 33-1
오늘1307 가재울점 남가좌동,5.030577,서대문구,서울특별시 서대문구 가재울미래로 2
히어로보드게임카페 신촌점 창천동,4.956775,서대문구,서울특별시 서대문구 명물길 23
카페수에르떼 홍제동,4.919868,서대문구,서울특별시 서대문구 세검정로3길 72-10
헤일우드 홍제동,4.817661,서대문구,서울특별시 서대문구 통일로34길 7
히자우 홍제동,4.815453,서대문구,서울특별시 서대문구 세검정로4길 6
증가로커피공방 남가좌동,4.783660,서대문구,서울특별시 서대문구 증가로10길 36-55
미네르바 창천동,4.762289,서대문구,서울특별시 서대문구 명물길 22
